## Setup

In [1]:
%pip install matplotlib wurlitzer ninja numba numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os,math,sys,re
import numpy as np
import torch
from types import SimpleNamespace as ns
from collections import namedtuple

from utils import show_img,load_cuda,cuda_begin,cdiv

In [3]:
np.set_printoptions(precision=2, linewidth=140)
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)

In [4]:
def cdiv(a,b):
    "Int ceiling division of `a` over `b`"
    return (a+b-1)//b

In [5]:
%load_ext wurlitzer

In [6]:
# os.environ['CUDA_LAUNCH_BLOCKING']='1'
# os.environ['NUMBA_ENABLE_CUDASIM'] = '1'

In [7]:
from numba import cuda
import torch

@cuda.jit
def sigmoid_forward(input, input_len, out):
    cbi,cbd,tid = cuda.blockIdx,cuda.blockDim,cuda.threadIdx
    idx = cbi.x * cbd.x + tid.x

    if idx >= input_len:
        return
    
    if input[idx] >= 0:
        res = 1. / ( 1. + math.exp(-input[idx]) )
    else:
        res = math.exp(input[idx]) / ( 1. + math.exp(input[idx]) )

    out[idx] = res


@cuda.jit
def sigmoid_backward(input, input_len, out):
    cbi,cbd,tid = cuda.blockIdx,cuda.blockDim,cuda.threadIdx
    idx = cbi.x * cbd.x + tid.x

    if idx >= input_len:
        return
    
    out[idx] = input[idx]*(1-input[idx])


def sigmoid_forward_torch(input):
    out_tensor = torch.empty_like(input)
    positive_mask = input >= 0
    out_tensor[positive_mask] = 1. / (1. + torch.exp(-input[positive_mask]))
    out_tensor[~positive_mask] = torch.exp(input[~positive_mask]) / (1. + torch.exp(input[~positive_mask]))
    
    return out_tensor

def sigmoid_backward_torch(input):
    return input * (1 - input)

In [8]:
def sigmoid_numba(input, fun, tw=16, gradcheck=False):
    (input_len,) = input.shape
    out = torch.zeros(input_len, dtype=torch.float32)
    out = out.contiguous().cuda()
    tpb = tw
    blocks = cdiv(input_len,tpb)
    fun[blocks, tpb](input, input_len, out) 
    return out
    

In [9]:
import torch

input = torch.as_tensor([0.3, -100000, 100000, 0.5, -0.5], dtype=torch.float32)
input = input.contiguous().cuda()

res = sigmoid_numba(input, sigmoid_forward, 1)
res

/opt/conda/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 5 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


tensor([0.57, 0.00, 1.00, 0.62, 0.38], device='cuda:0')

In [10]:
res = sigmoid_numba(res, sigmoid_backward, 1)
res

/opt/conda/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 5 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


tensor([0.24, 0.00, 0.00, 0.24, 0.24], device='cuda:0')

## Interface for Torch Gradtest

Pytorch has an awesome function that does numerical differentiation and checks if our custom backward pass is correct. To use it we must encapsulate our code within torch's autograd

In [11]:
class GradcheckSigmoid(torch.autograd.Function):
    @staticmethod
    def forward(ctx, data: torch.Tensor) -> torch.Tensor:
        result = sigmoid_forward_torch(data)
        ctx.save_for_backward(result)
        return result

    @staticmethod
    def backward(ctx, grad_output: torch.Tensor) -> torch.Tensor:
        (result,) = ctx.saved_tensors
        grad = sigmoid_backward_torch(result)
        return grad_output * grad

In [12]:
import torch

torch.manual_seed(42)

sigmoid = GradcheckSigmoid.apply
data = torch.randn(4, dtype=torch.double, requires_grad=True)

# `torch.autograd.gradcheck` takes a tuple of tensors as input, check if your gradient evaluated
# with these tensors are close enough to numerical approximations and returns `True` if they all
# verify this condition.
if torch.autograd.gradcheck(sigmoid, data, eps=1e-8, atol=1e-7):
    print("gradcheck successful")
else:
    print("gradcheck unsuccessful")

gradcheck successful


## Converting NUMBA code to C CUDA

I've used the following request to chat GPT 3.5: "Convert the following 2 CUDA numba python functions to 2 C CUDA functions. Do the minimal amount of changes."

And then added the sigmoid_forward and sigmoid_backward python code.

In [13]:
cuda_fwd_src = cuda_begin + r"""
#include <math.h>

__global__ void sigmoid_forward_cuda_kernel(const float* input, int input_len, float* out) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < input_len) {
        float res;
        if (input[idx] >= 0) {
            res = 1. / (1. + expf(-input[idx]));
        } else {
            res = expf(input[idx]) / (1. + expf(input[idx]));
        }

        out[idx] = res;
    }
}

torch::Tensor sigmoid_forward_cuda(torch::Tensor input) {
    CHECK_INPUT(input);
    // Get the data pointers and sizes
    float* input_data_ptr = input.data_ptr<float>();
    int input_len = input.numel();

    // Allocate output tensor on GPU
    torch::Tensor out_tensor = torch::empty_like(input);

    // Get the data pointer for the output tensor
    float* out_data_ptr = out_tensor.data_ptr<float>();

    // Set block and grid dimensions
    int threads_per_block = 256; // You may adjust this based on your specific GPU capabilities
    int num_blocks = (input_len + threads_per_block - 1) / threads_per_block;

    // Launch CUDA kernel
    sigmoid_forward_cuda_kernel<<<num_blocks, threads_per_block>>>(input_data_ptr, input_len, out_data_ptr);

    // Synchronize to ensure the kernel is done before proceeding
    cudaDeviceSynchronize();
    C10_CUDA_KERNEL_LAUNCH_CHECK();

    return out_tensor;
}
"""

In [14]:
cuda_bwd_src = cuda_begin + r"""
__global__ void sigmoid_backward_cuda_kernel(const float* input, int input_len, float* out) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < input_len) {
        out[idx] = input[idx] * (1 - input[idx]);
    }
}

torch::Tensor sigmoid_backward_cuda(torch::Tensor input_tensor) {
    // Ensure the input is a float tensor on the GPU
    torch::Tensor input_tensor_cuda = input_tensor.cuda().to(torch::kFloat);

    // Get the data pointers and sizes
    const float* input_data_ptr = input_tensor_cuda.data_ptr<float>();
    int input_len = input_tensor_cuda.numel();

    // Allocate output tensor on GPU
    torch::TensorOptions options = torch::TensorOptions().device(torch::kCUDA).dtype(torch::kFloat);
    torch::Tensor out_tensor = torch::empty({input_len}, options);

    // Get the data pointer for the output tensor
    float* out_data_ptr = out_tensor.data_ptr<float>();

    // Set block and grid dimensions
    int threads_per_block = 256; // You may adjust this based on your specific GPU capabilities
    int num_blocks = (input_len + threads_per_block - 1) / threads_per_block;

    // Launch CUDA kernel
    sigmoid_backward_cuda_kernel<<<num_blocks, threads_per_block>>>(input_data_ptr, input_len, out_data_ptr);

    // Synchronize to ensure the kernel is done before proceeding
    cudaDeviceSynchronize();

    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return out_tensor;
}
"""

In [15]:
fname = 'sigmoid_forward_cuda'
cpp_src = 'torch::Tensor sigmoid_forward_cuda(torch::Tensor input);'

In [16]:
module_forward = load_cuda(cuda_fwd_src, cpp_src, [fname])

In [17]:
res = module_forward.sigmoid_forward_cuda(input)
res

tensor([0.57, 0.00, 1.00, 0.62, 0.38], device='cuda:0')

In [18]:
fname = 'sigmoid_backward_cuda'
cpp_src = 'torch::Tensor sigmoid_backward_cuda(torch::Tensor input);'
# cuda_bwd_src
module_backward = load_cuda(cuda_bwd_src, cpp_src, [fname])

In [19]:
grad = module_backward.sigmoid_backward_cuda(res)
grad

tensor([0.24, 0.00, 0.00, 0.24, 0.24], device='cuda:0')

## Check our CUDA Sigmoid gradients

In [20]:
class GradcheckSigmoid(torch.autograd.Function):
    @staticmethod
    def forward(ctx, data: torch.Tensor) -> torch.Tensor:
        result = module_forward.sigmoid_forward_cuda(data)
        ctx.save_for_backward(result)
        return result

    @staticmethod
    def backward(ctx, grad_output: torch.Tensor) -> torch.Tensor:
        (result,) = ctx.saved_tensors
        grad = module_backward.sigmoid_backward_cuda(result)
        return grad_output * grad

In [21]:
import torch

torch.manual_seed(42)

sigmoid = GradcheckSigmoid.apply
data = torch.randn(4, dtype=torch.float, requires_grad=True).contiguous().cuda()

# Changing eps and atol since we are dealing with float32
if torch.autograd.gradcheck(sigmoid, data, eps=5e-4, atol=1e-7):
    print('gradcheck successful')
else:
    print('gradcheck unsuccessful')

gradcheck successful


/opt/conda/lib/python3.10/site-packages/torch/autograd/gradcheck.py:915: UserWarning: Input #0 requires gradient and is not a double precision floating point or complex. This check will likely fail if all the inputs are not of double precision floating point or complex. 
  warnings.warn(
